# Import Materials
This example shows how to import materials.

### Import the required packages

In [1]:
import json
from pathlib import Path
import tempfile

from IPython.display import display
from ansys.aedt.core.downloads import download_file
import pandas as pd

from pyedb import Edb

AEDT_VERSION = "2024.2"
NG_MODE = False


Download the example PCB data.

In [2]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", destination=temp_folder.name)

## Load example layout.

In [3]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.34.3


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.2


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


## Review materials from layout

Get materials from layout in a dictionary. Materials are exported together with stadckup.

In [4]:
data_cfg = edbapp.configuration.get_data_from_db(stackup=True)

PyAEDT INFO: Getting data from layout database.


In [5]:
df = pd.DataFrame(data=data_cfg["stackup"]["materials"])
display(df)

,name,permittivity,conductivity,dielectric_loss_tangent,magnetic_loss_tangent,mass_density,permeability,poisson_ratio,specific_heat,thermal_conductivity
0,copper,0.00,58000000.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000
1,FR4_epoxy,4.40,0.0,0.020,0.0,1900.0,0.0,0.28,1150.0,0.294
2,Megtron4,3.77,0.0,0.005,0.0,0.0,0.0,0.00,0.0,0.000
3,Megtron4_2,3.47,0.0,0.006,0.0,0.0,0.0,0.00,0.0,0.000
4,Megtron4_3,4.20,0.0,0.005,0.0,0.0,0.0,0.00,0.0,0.000
5,Solder Resist,3.00,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000


## Add a new material

In [6]:
data_cfg["stackup"]["materials"].append(
    {"name": "soldermask", "permittivity": 3.3, "dielectric_loss_tangent": 0.02},
)

## Edit existing material properties

In [7]:
data_cfg["stackup"]["materials"][1]["name"] = "fr4_epoxy"
data_cfg["stackup"]["materials"][1]["dielectric_loss_tangent"] = 0.015

## Review modified materials

In [8]:
df = pd.DataFrame(data=data_cfg["stackup"]["materials"])
display(df)

,name,permittivity,conductivity,dielectric_loss_tangent,magnetic_loss_tangent,mass_density,permeability,poisson_ratio,specific_heat,thermal_conductivity
0,copper,0.00,58000000.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000
1,fr4_epoxy,4.40,0.0,0.015,0.0,1900.0,0.0,0.28,1150.0,0.294
2,Megtron4,3.77,0.0,0.005,0.0,0.0,0.0,0.00,0.0,0.000
3,Megtron4_2,3.47,0.0,0.006,0.0,0.0,0.0,0.00,0.0,0.000
4,Megtron4_3,4.20,0.0,0.005,0.0,0.0,0.0,0.00,0.0,0.000
5,Solder Resist,3.00,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000
6,soldermask,3.30,NaN,0.020,NaN,NaN,NaN,NaN,NaN,NaN


## Write material definition into a json file

In [9]:
file_cfg = Path(temp_folder.name) / "edb_configuration.json"
with open(file_cfg, "w") as f:
    json.dump(data_cfg, f, indent=4, ensure_ascii=False)

## Load materials from json configuration file

In [10]:
edbapp.configuration.load(str(file_cfg), apply_file=True)

## Review materials from layout

In [11]:
edbapp.materials.materials

{'copper': <pyedb.dotnet.edb_core.materials.Material at 0x2d69cd548b0>,
 'fr4_epoxy': <pyedb.dotnet.edb_core.materials.Material at 0x2d6f575e080>,
 'Megtron4': <pyedb.dotnet.edb_core.materials.Material at 0x2d6a0dc2ad0>,
 'Megtron4_2': <pyedb.dotnet.edb_core.materials.Material at 0x2d6a0dc2b30>,
 'Megtron4_3': <pyedb.dotnet.edb_core.materials.Material at 0x2d6a0dc2bc0>,
 'Solder Resist': <pyedb.dotnet.edb_core.materials.Material at 0x2d6a0dc2b90>,
 'soldermask': <pyedb.dotnet.edb_core.materials.Material at 0x2d6a0dc2a70>}

## Check modified material properties

In [12]:
edbapp.materials["fr4_epoxy"].loss_tangent

0.015

## Save and close Edb
The temporary folder will be deleted once the execution of this script is finished. Replace **edbapp.save()** with
**edbapp.save_as("C:/example.aedb")** to keep the example project.

In [13]:
edbapp.save()
edbapp.close()

True